In [1]:
import pandas as pd
import numpy as np
import random
import keras
import matplotlib.pyplot as plt
import seaborn as sns

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [2]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix
from keras.utils import plot_model
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_curve
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [3]:
df1 = pd.read_csv("/kaggle/input/nslkdd/kdd_train.csv")
df2 = pd.read_csv("/kaggle/input/nslkdd/kdd_test.csv")

In [4]:
df = pd.concat([df1,df2])
del df1,df2

In [5]:
df.describe(include="all")

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,labels
count,148517.000000,148517,148517,148517,1.485170e+05,1.485170e+05,148517.000000,148517.000000,148517.000000,148517.000000,...,148517.000000,148517.000000,148517.000000,148517.000000,148517.000000,148517.000000,148517.000000,148517.000000,148517.000000,148517
unique,NaN,3,70,11,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37
top,NaN,tcp,http,SF,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,normal
freq,NaN,121197,47645,88748,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78588
mean,282.850980,NaN,NaN,NaN,4.051230e+04,1.722710e+04,0.000182,0.021822,0.000121,0.196994,...,116.787910,0.524935,0.083248,0.147277,0.031895,0.276092,0.270477,0.124323,0.125298,NaN
std,2545.220043,NaN,NaN,NaN,5.409528e+06,3.703596e+06,0.013482,0.248406,0.014678,2.087903,...,110.868085,0.448749,0.190616,0.308485,0.111598,0.440218,0.441200,0.312011,0.324583,NaN
min,0.000000,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
25%,0.000000,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,11.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
50%,0.000000,NaN,NaN,NaN,4.400000e+01,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,65.000000,0.540000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
75%,0.000000,NaN,NaN,NaN,2.760000e+02,5.270000e+02,0.000000,0.000000,0.000000,0.000000,...,255.000000,1.000000,0.070000,0.060000,0.020000,1.000000,1.000000,0.000000,0.000000,NaN


In [6]:
df['labels'] = df['labels'].map(lambda a: 0 if a == 'normal' else 1)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 148517 entries, 0 to 22543
Data columns (total 42 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   duration                     148517 non-null  int64  
 1   protocol_type                148517 non-null  object 
 2   service                      148517 non-null  object 
 3   flag                         148517 non-null  object 
 4   src_bytes                    148517 non-null  int64  
 5   dst_bytes                    148517 non-null  int64  
 6   land                         148517 non-null  int64  
 7   wrong_fragment               148517 non-null  int64  
 8   urgent                       148517 non-null  int64  
 9   hot                          148517 non-null  int64  
 10  num_failed_logins            148517 non-null  int64  
 11  logged_in                    148517 non-null  int64  
 12  num_compromised              148517 non-null  int64  
 13  

In [8]:
unique_values = df['num_outbound_cmds'].unique()
unique_values

array([0])

In [9]:
df.drop('num_outbound_cmds', axis=1, inplace=True)

In [10]:
df.isnull().sum().sort_values(ascending = False)

duration                       0
count                          0
serror_rate                    0
srv_serror_rate                0
rerror_rate                    0
srv_rerror_rate                0
same_srv_rate                  0
diff_srv_rate                  0
srv_diff_host_rate             0
dst_host_count                 0
dst_host_srv_count             0
dst_host_same_srv_rate         0
dst_host_diff_srv_rate         0
dst_host_same_src_port_rate    0
dst_host_srv_diff_host_rate    0
dst_host_serror_rate           0
dst_host_srv_serror_rate       0
dst_host_rerror_rate           0
dst_host_srv_rerror_rate       0
srv_count                      0
is_guest_login                 0
protocol_type                  0
is_host_login                  0
service                        0
flag                           0
src_bytes                      0
dst_bytes                      0
land                           0
wrong_fragment                 0
urgent                         0
hot       

In [11]:
df.duplicated().sum()

15801

In [12]:
df = df.drop_duplicates()

In [13]:
df.describe(include="all")

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,labels
count,132716.000000,132716,132716,132716,1.327160e+05,1.327160e+05,132716.000000,132716.000000,132716.000000,132716.000000,...,132716.000000,132716.00000,132716.000000,132716.000000,132716.000000,132716.000000,132716.000000,132716.000000,132716.000000,132716.000000
unique,NaN,3,70,11,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,tcp,http,SF,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,108426,42697,79386,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,284.435803,NaN,NaN,NaN,4.402288e+04,1.890656e+04,0.000188,0.021949,0.000128,0.199064,...,116.922692,0.52545,0.083365,0.146808,0.031675,0.275095,0.269514,0.124585,0.125523,0.470222
std,2560.779464,NaN,NaN,NaN,5.721870e+06,3.917790e+06,0.013724,0.249221,0.015283,2.099599,...,110.869672,0.44860,0.190763,0.307950,0.110427,0.439641,0.440631,0.312059,0.324732,0.499114
min,0.000000,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,11.000000,0.05000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,NaN,NaN,NaN,4.400000e+01,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,65.000000,0.54000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,NaN,NaN,NaN,2.760000e+02,5.340000e+02,0.000000,0.000000,0.000000,0.000000,...,255.000000,1.00000,0.070000,0.060000,0.020000,1.000000,1.000000,0.000000,0.000000,1.000000


In [14]:
le_protocol_type = LabelEncoder()
le_service = LabelEncoder()
le_flag = LabelEncoder()

In [15]:
df['protocol_type_encoded'] = le_protocol_type.fit_transform(df.protocol_type)
df['service_encoded'] = le_service.fit_transform(df.service)
df['flag_encoded'] = le_flag.fit_transform(df.flag)

In [16]:
protocol_type_ = OneHotEncoder()
service_ = OneHotEncoder()
flag_ = OneHotEncoder()

In [17]:
X = protocol_type_.fit_transform(df.protocol_type_encoded.values.reshape(-1,1)).toarray()
Xm = service_.fit_transform(df.service_encoded.values.reshape(-1,1)).toarray()
Xmm = flag_.fit_transform(df.flag_encoded.values.reshape(-1,1)).toarray()

In [18]:
# Assuming you have DataFrames: df, dfOneHot, dfOneHot2, dfOneHot3

dfOneHot = pd.DataFrame(X, columns=["protocol_type_"+str(int(i)) for i in range(X.shape[1])])
df = pd.concat([df.reset_index(drop=True), dfOneHot], axis=1)

dfOneHot2 = pd.DataFrame(Xm, columns=["service_"+str(int(i)) for i in range(Xm.shape[1])])
df = pd.concat([df.reset_index(drop=True), dfOneHot2], axis=1)

dfOneHot3 = pd.DataFrame(Xmm, columns=["flag_"+str(int(i)) for i in range(Xmm.shape[1])])
df = pd.concat([df.reset_index(drop=True), dfOneHot3], axis=1)

In [19]:
df.drop(columns=['protocol_type','service','flag'], inplace = True )
df.shape

(132716, 125)

In [20]:
# Feature Selection
bestfeatuers=SelectKBest(score_func=chi2,k=100)
inp=df.iloc[:,0:125]
fit=bestfeatuers.fit(inp,df['labels'])
dfscores=pd.DataFrame(fit.scores_)
dfcol=pd.DataFrame(inp.columns)
featurescore=pd.concat([dfcol,dfscores],axis=1)
featurescore.columns=['feature','score']
k=featurescore['feature']
t=featurescore.nlargest(100,'score')
li=list(t['feature'])
print("Top 100 features:")
#print("\n")
for x in li:
    print(x,end=", ")

Top 100 features:
src_bytes, dst_bytes, duration, dst_host_srv_count, count, dst_host_count, service_encoded, labels, flag_encoded, dst_host_srv_serror_rate, flag_5, srv_serror_rate, serror_rate, dst_host_serror_rate, logged_in, num_root, flag_9, service_24, num_compromised, dst_host_same_srv_rate, service_49, same_srv_rate, srv_rerror_rate, rerror_rate, service_12, dst_host_srv_rerror_rate, dst_host_rerror_rate, protocol_type_2, service_54, flag_1, protocol_type_0, dst_host_diff_srv_rate, service_14, wrong_fragment, diff_srv_rate, service_15, flag_4, protocol_type_encoded, srv_diff_host_rate, flag_2, srv_count, num_file_creations, service_2, service_20, service_66, service_6, service_5, service_28, service_67, service_69, service_29, service_41, service_68, service_8, service_58, dst_host_same_src_port_rate, service_7, service_10, service_26, service_9, service_21, service_16, service_17, service_33, service_59, service_65, service_36, service_23, service_11, service_35, service_60, s

In [21]:
X=df[li].drop(columns=['labels'])
y=df[li]['labels']

In [22]:
# Split the data into train (80%) and test (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Split the training data further into train (80%) and validation (20%) sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.125, random_state=42)

In [23]:
print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)
print("X_test shape:", X_test.shape)

X_train shape: (92900, 99)
X_val shape: (13272, 99)
X_test shape: (26544, 99)


In [24]:
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [25]:
# Make sure to replace input_shape and output_classes with the correct values for your dataset
input_shape = (X_train.shape[1], 1)
output_classes = 1

In [26]:
def create_1d_cnn_model(params):
    model = Sequential()
    
    model.add(Conv1D(filters=params['num_filters'], kernel_size=params['kernel_size'], activation='relu', input_shape=(input_shape)))
    model.add(MaxPooling1D(pool_size=params['pooling_size']))
    
    model.add(Conv1D(filters=params['num_filters'], kernel_size=params['kernel_size'], activation='relu'))
    model.add(MaxPooling1D(pool_size=params['pooling_size']))
       
    model.add(Dropout(params['dropout_rate']))    
    model.add(Flatten())

    for _ in range(params['num_dense_layers']):
        model.add(Dense(params['num_neurons_in_dense_layers'], activation='relu'))
        model.add(Dropout(params['dropout_rate']))

    model.add(Dense(output_classes, activation='sigmoid'))

    optimizer = Adam(learning_rate=params['learning_rate'])
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

In [27]:
# Defines the number of particles in the swarm and the number of iterations
n_particles = 20
n_iterations = 10

# Defines the cognitive and social constants
c1 = 2
c2 = 2

# Define the boundaries of the search space (min and max values for each parameter)
bounds = [(0, 4), (0, 4), (2, 6), (1, 5), (0, 4), (0.1, 0.5), (1e-5, 1e-2), (0, 4), (10, 100)]

# Initializes the particles at random positions within the search space
particles = [np.random.uniform(low=bounds[i][0], high=bounds[i][1], size=n_particles) for i in range(len(bounds))]

# Initializes the velocities of the particles to zero
velocities = [np.zeros(n_particles) for _ in range(len(bounds))]

In [28]:
def position_to_params(particles, i):
    params_keys = ['num_filters', 'kernel_size', 'pooling_size', 'num_dense_layers', 'num_neurons_in_dense_layers', 'dropout_rate', 'learning_rate', 'batch_size', 'num_epochs']

    params = {}
    for j, key in enumerate(params_keys):
        # Values are already initialized in the correct range, no need to scale
        value = particles[j][i]
        # If this parameter should be an integer, round the value
        if key in ['num_filters', 'kernel_size', 'pooling_size', 'num_dense_layers', 'num_neurons_in_dense_layers', 'batch_size', 'num_epochs']:
            value = int(round(value))
        
        # Map the continuous values to the discrete values for certain parameters
        if key == 'num_filters' or key == 'num_neurons_in_dense_layers' or key == 'batch_size':
            value_choices = [16, 32, 64, 128, 256]
            value = value_choices[min(int(value), len(value_choices)-1)]
        elif key == 'kernel_size':
            value_choices = [3, 5, 7, 9, 11]
            value = value_choices[min(int(value), len(value_choices)-1)]
        
        params[key] = value
    return params

In [29]:
# Initializes the personal best positions and the global best position
pbest_positions = [position_to_params(particles, i) for i in range(n_particles)]

#pbest_positions = particles
pbest_fitnesses = [0] * n_particles
gbest_position = None
gbest_fitness = None

In [30]:
def evaluate_fitness(params):
    model = create_1d_cnn_model(params)
    
    # Create a custom callback to print epoch progress
    class PrintEpochProgress(keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs=None):
            print(f"  Epoch {epoch+1}/{params['num_epochs']} - val_accuracy: {logs['val_accuracy']:.4f}")
    
    history = model.fit(X_train, y_train, batch_size=params['batch_size'], epochs=params['num_epochs'], verbose=0, validation_data=(X_val, y_val), callbacks=[PrintEpochProgress()])
    return history.history['val_accuracy'][-1]

In [ ]:
params_keys = ['num_filters', 'kernel_size', 'pooling_size', 'num_dense_layers', 'num_neurons_in_dense_layers', 'dropout_rate', 'learning_rate', 'batch_size', 'num_epochs']

for iteration in range(n_iterations):
    for i in range(n_particles):
        # Converts the position of the particle to a set of parameters
        params = position_to_params(particles, i)

        # Evaluates the fitness of the particle
        fitness = evaluate_fitness(params)                 
            
        # Updates the personal best position of the particle
        if fitness > pbest_fitnesses[i]:
            pbest_fitnesses[i] = fitness
            pbest_positions[i] = params  # Store the parameters, not the position

        # Updates the global best position
        if gbest_fitness is None or fitness > gbest_fitness:
            gbest_fitness = fitness
            gbest_position = params  # Store the parameters, not the position            
            
    # Updates the velocities and positions of the particles
    for i in range(n_particles):
        for j, key in enumerate(params_keys): # params_keys should be the list of parameter names
            r1, r2 = np.random.random(), np.random.random()
            velocities[j][i] = 0.5 * velocities[j][i] + c1 * r1 * (pbest_positions[i][key] - particles[j][i]) + c2 * r2 * (gbest_position[key] - particles[j][i])
            particles[j][i] += velocities[j][i]
            # Clip the updated parameters to be within their bounds
            particles[j][i] = np.clip(particles[j][i], bounds[j][0], bounds[j][1])

  Epoch 1/70 - val_accuracy: 0.9788
  Epoch 2/70 - val_accuracy: 0.9807
  Epoch 3/70 - val_accuracy: 0.9853
  Epoch 4/70 - val_accuracy: 0.9870
  Epoch 5/70 - val_accuracy: 0.9861
  Epoch 6/70 - val_accuracy: 0.9888
  Epoch 7/70 - val_accuracy: 0.9894
  Epoch 8/70 - val_accuracy: 0.9904
  Epoch 9/70 - val_accuracy: 0.9904
  Epoch 10/70 - val_accuracy: 0.9921
  Epoch 11/70 - val_accuracy: 0.9910
  Epoch 12/70 - val_accuracy: 0.9916
  Epoch 13/70 - val_accuracy: 0.9908
  Epoch 14/70 - val_accuracy: 0.9913
  Epoch 15/70 - val_accuracy: 0.9915
  Epoch 16/70 - val_accuracy: 0.9917
  Epoch 17/70 - val_accuracy: 0.9925
  Epoch 18/70 - val_accuracy: 0.9917
  Epoch 19/70 - val_accuracy: 0.9924
  Epoch 20/70 - val_accuracy: 0.9929


In [ ]:
best_parameters = gbest_position

In [ ]:
print("Best parameters found:")
for key, value in best_parameters.items():
    print(f"{key}: {value}")

In [ ]:
best_model = create_1d_cnn_model(best_parameters)
history = best_model.fit(X_train, y_train, batch_size=best_parameters['batch_size'], epochs=best_parameters['num_epochs'], validation_data=(X_val, y_val))

In [ ]:
# Plot the training and validation accuracy over the epochs
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()
plt.savefig('acc_plo.jpg')

In [ ]:
# Plot the training and validation loss over the epochs
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()
plt.savefig('los_plo.jpg')

In [ ]:
# Print train, test and validation accuracy
train_loss, train_acc = best_model.evaluate(np.expand_dims(X_train, axis=2), y_train, verbose=0)
print('Train Loss: {:.2f}%'.format(train_loss*100))
print('Train Accuracy: {:.2f}%'.format(train_acc*100))

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = best_model.evaluate(np.expand_dims(X_test, axis=2), y_test, verbose=0)
print("\nModel performance on test dataset:")
print("Test Loss: {:.2f}%".format(test_loss*100))
print("Test Accuracy: {:.2f}%".format(test_accuracy*100))

In [ ]:
plot_model(best_model, to_file='model.png', show_shapes=True, show_layer_names=True)

In [ ]:
# Generate predictions on the test set
y_pred = best_model.predict(np.expand_dims(X_test, axis=2))

In [ ]:
# Performance comparison table
print(classification_report(y_test, np.round(y_pred)))

In [ ]:
# Compute the confusion matrix
conf_mat = confusion_matrix(y_test, np.round(y_pred))

# Define the class labels
class_labels = ['No Intrusion', 'Intrusion']

# Create a heatmap plot of the confusion matrix
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)

# Set the plot labels and title
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')

# Show the plot
plt.show()
plt.savefig('con_max.jpg')

In [ ]:
# Predict the test set
y_pred = best_model.predict(np.expand_dims(X_test, axis=2))
y_pred = (y_pred > 0.5)

# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# Print confusion matrix as heatmap with percentages
plt.figure(figsize=(8,6))
sns.heatmap(cm_norm, annot=True, cmap='Blues', fmt='.2f')
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix with Percentages')
plt.show()
plt.savefig('con_muxf.jpg')

In [ ]:
# ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], linestyle='--')
plt.title("ROC Curve")
plt.show()
plt.savefig('roc_cur.jpg')

In [ ]:
# Precision-Recall curve
precision, recall, thresholds = precision_recall_curve(y_test, y_pred)
plt.plot(recall, precision)
plt.title("Precision-Recall Curve")
plt.show()
plt.savefig('pre_rec.jpg')

In [ ]:
# Get the test accuracy and loss for each epoch from the history object
test_acc = history.history['val_accuracy']
test_loss = history.history['val_loss']

# Plot the test accuracy and loss
plt.plot(test_acc, label='Test Accuracy')
plt.plot(test_loss, label='Test Loss')

# Add axis labels and a legend
plt.xlabel('Epoch')
plt.ylabel('Accuracy/Loss')
plt.title('Test Accuracy and Loss')
plt.legend()

# Show the plot
plt.show()
plt.savefig('tes_los.jpg')